# and then we go to mario draghi

In [1]:
import pandas as pd
import time
import datetime
import matplotlib
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
import math
import requests
from bs4 import BeautifulSoup as bs
import pickle as pk

In [2]:
ad = pd.read_csv('datasets/1976-2020-president.csv')
years = set(ad['year'])
states = set(ad['state'])
dflist = list()

In [3]:
states.remove('DISTRICT OF COLUMBIA')
years.remove(1976)
years.remove(1980)

#### some function

In [4]:
def foofloat(x):
    try:
        return float(x)
    except:
        return np.NaN

In [5]:
def bls_serial(series_id):
  bsobj = bs(requests.post("https://data.bls.gov/pdq/SurveyOutputServlet", {'series_id': series_id}).text)
  return bsobj.findAll('table')[0].findAll('tr')[2].findAll('td')[0].text.strip().upper()

In [6]:
def ecdf_inv(a, q): return sorted(a)[int(len(a) * q)]

In [7]:
def to_percent(older, col_key, shift=0):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y-shift, s, 'DEMOCRAT'][col_key]
                r = older.loc[y-shift, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [8]:
def to_splatter(a, col_name): # from row-column to table-list
    lst = []
    for y in a.columns[2:]:
        for index, row in a.iterrows():
            lst.append([int(y), row['GeoName'].upper(), row[y], ])
    
    return pd.DataFrame(data=lst, columns=['year', 'state', col_name, ])

In [9]:
def var42(df_res, colname, filename, old_colname):
    new_df = []
    
    df_res = df_res.set_index(['year', 'state'])
    for y in years:
        for s in states:
            try:
                new_df.append([y,s,] + [df_res.loc[y-i,s][old_colname] for i in range(4)])
            except KeyError:
                pass

    new_df = pd.DataFrame(data=new_df, columns = ['year', 'state',] + [f'{colname}_{i}' for i in range(4,0,-1)])
    new_df = new_df.sort_values(by=['year', 'state'])
    new_df.to_csv(f'datasets-clean/{filename}.csv', index=False)
    return new_df

In [10]:
def beadiff(rowcol, colname):
    difflist = []

    for s in states:
        tmp = rowcol.loc[(slice(None),s),:]
        yyyy = list({i[0] for i in tmp.index})
        yyyy.sort()
        for y in yyyy[1:]:
            q = rowcol.loc[(y,s),:][colname]/rowcol.loc[(int(y)-1,s),:][colname]
            difflist.append([y, s, (q - 1)*100])
            
    return  pd.DataFrame(data=difflist, columns=['year', 'state', colname])

#### y_hat aka presitdent popular vote

In [11]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote = pop_vote.rename(columns={"candidatevotes": "y_votes_percent", })
pop_vote = pop_vote.sort_values(by=['year', 'state'])
pop_vote.to_csv('datasets-clean/popular-vote-y.csv', index=False)
dflist.append(pop_vote)
pop_vote

,year,state,y_votes_percent
43,1984,ALABAMA,38.736605
5,1984,ALASKA,30.944087
15,1984,ARIZONA,32.883272
27,1984,ARKANSAS,38.772412
22,1984,CALIFORNIA,41.775465
...,...,...,...
188,2020,VIRGINIA,55.154687
194,2020,WASHINGTON,59.925503
164,2020,WEST VIRGINIA,30.201468
198,2020,WISCONSIN,50.319063


#### GDP and friends

It was necessary to generate an adjusted series of state GDP because of a change in BEA’s estimation procedure from a Standard Industrial Classification (SIC) basis to a North American Industry Classification System (NAICS) basis in 1997.
Data prior to 1997 were adjusted to avoid any erratic shifts in GDP that year.
While the change to NAICS basis occurred in 1997, BEA also provides estimates under a SIC basis in that year.
~~Our adjustment involved calculating the 1997 ratio of NAICS-based GDP to SIC-based GDP for each state, and multiplying it by SIC-based GDP in all years prior to 1997 to obtain our adjusted series of state-level GDP.~~

In [12]:
gdp_nom_97 = to_splatter(pd.read_csv('datasets/gdp-nominal-63-97.csv'), 'nominal_gdp_mln')
gdp_nom_12 = to_splatter(pd.read_csv('datasets/gdp-nominal-97-20.csv'), 'nominal_gdp_mln')
gdp_real_97 = to_splatter(pd.read_csv('datasets/gdp-real-77-97-chain-97.csv'), 'gdp_real')
gdp_real_12 = to_splatter(pd.read_csv('datasets/gdp-real-97-20-chain-12.csv'), 'gdp_real')

In [13]:
gdp_nom_97 = gdp_nom_97.set_index(['year', 'state'])
gdp_nom_12 = gdp_nom_12.set_index(['year', 'state'])
gdp_real_97 = gdp_real_97.set_index(['year', 'state'])
gdp_real_12 = gdp_real_12.set_index(['year', 'state'])

In [14]:
gdp_def_97 = (gdp_nom_97['nominal_gdp_mln'] / gdp_real_97['gdp_real']).to_frame('gdp_def').dropna()
gdp_def_12 = (gdp_nom_12['nominal_gdp_mln'] / gdp_real_12['gdp_real']).to_frame('gdp_def').dropna()

In [15]:
gdp_def = pd.concat([beadiff(gdp_def_12,'gdp_def'), beadiff(gdp_def_97,'gdp_def')])
gdp_def = gdp_def.sort_values(by=['year', 'state'])
gdp_def

,year,state,gdp_def
860,1978,ALABAMA,7.402566
100,1978,ALASKA,11.065835
300,1978,ARIZONA,7.468516
540,1978,ARKANSAS,8.054511
440,1978,CALIFORNIA,7.094885
...,...,...,...
896,2020,VIRGINIA,1.950355
1034,2020,WASHINGTON,1.658615
344,2020,WEST VIRGINIA,-0.257632
1126,2020,WISCONSIN,2.055099


In [16]:
def_index = var42(gdp_def, 'def', 'price-index', 'gdp_def')
dflist.append(def_index)
def_index

,year,state,def_4,def_3,def_2,def_1
43,1984,ALABAMA,4.548485,4.185139,6.426162,9.372111
5,1984,ALASKA,1.250442,-1.018197,4.758325,23.893438
15,1984,ARIZONA,4.731524,5.112800,6.966784,8.376158
27,1984,ARKANSAS,4.363585,4.161762,5.603037,8.995826
22,1984,CALIFORNIA,4.737549,4.477916,6.742430,8.871767
...,...,...,...,...,...,...
188,2020,VIRGINIA,1.950355,2.222617,1.866446,1.307918
194,2020,WASHINGTON,1.658615,1.894215,1.658639,1.276351
164,2020,WEST VIRGINIA,-0.257632,0.908538,3.283808,3.326667
198,2020,WISCONSIN,2.055099,2.222565,1.889366,1.230655


In [17]:
gdp_real = pd.concat([beadiff(gdp_real_97,'gdp_real'), beadiff(gdp_real_12,'gdp_real')])
gdp_real = gdp_real.sort_values(by=['year', 'state'])
gdp_real

,year,state,gdp_real
860,1978,ALABAMA,6.421827
100,1978,ALASKA,8.958679
300,1978,ARIZONA,10.462245
540,1978,ARKANSAS,6.534163
440,1978,CALIFORNIA,6.881903
...,...,...,...
896,2020,VIRGINIA,-2.757118
1034,2020,WASHINGTON,-0.581850
344,2020,WEST VIRGINIA,-3.902932
1126,2020,WISCONSIN,-4.006500


#### z growth index

In [18]:
GDP_THRESHOLD = 2.67

In [19]:
z_yearly = gdp_real.copy()
z_yearly.gdp_real = z_yearly.gdp_real.apply(lambda x : 1 if x > GDP_THRESHOLD else 0)

growth_index = var42(z_yearly, 'z', 'z-growth-index', 'gdp_real')
dflist.append(growth_index)
growth_index

,year,state,z_4,z_3,z_2,z_1
43,1984,ALABAMA,1,1,0,0
5,1984,ALASKA,1,0,0,1
15,1984,ARIZONA,1,1,0,1
27,1984,ARKANSAS,1,1,0,1
22,1984,CALIFORNIA,1,1,0,1
...,...,...,...,...,...,...
188,2020,VIRGINIA,0,0,0,0
194,2020,WASHINGTON,0,1,1,1
164,2020,WEST VIRGINIA,0,0,0,0
198,2020,WISCONSIN,0,0,0,0


#### g gdp index

In [20]:
gdp_index = var42(gdp_real, 'gdp', 'gdp-growth-index', 'gdp_real')
dflist.append(gdp_index)
gdp_index

,year,state,gdp_4,gdp_3,gdp_2,gdp_1
43,1984,ALABAMA,4.719412,4.944172,-2.488787,1.840597
5,1984,ALASKA,3.995028,-2.715237,2.618149,15.266624
15,1984,ARIZONA,10.752575,5.715111,-2.352463,2.781647
27,1984,ARKANSAS,7.839078,3.206390,-2.673933,3.580354
22,1984,CALIFORNIA,8.028589,3.578332,0.028616,3.291989
...,...,...,...,...,...,...
188,2020,VIRGINIA,-2.757118,1.973924,2.377204,1.469457
194,2020,WASHINGTON,-0.581850,3.946746,6.794566,5.359424
164,2020,WEST VIRGINIA,-3.902932,-0.781380,2.315721,2.068235
198,2020,WISCONSIN,-4.006500,1.494643,2.430560,0.133906


#### incumbent

In [21]:
incumbent = pd.read_csv('datasets/incumbent-4president-76-20.csv', sep=';')

tmp = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
incumbent = pd.merge(tmp, incumbent, how='inner', left_on='year', right_on='year')
incumbent = incumbent.sort_values(by=['year', 'state'])
incumbent.to_csv('datasets-clean/incumbent-longitudinal-replication.csv', index=False)
dflist.append(incumbent)
incumbent

,year,state,incumbent,former_president_again,former_party_morethan_2,sudden_vice,lag_vice
43,1984,ALABAMA,-1,-1,0.0,0,0
5,1984,ALASKA,-1,-1,0.0,0,0
15,1984,ARIZONA,-1,-1,0.0,0,0
27,1984,ARKANSAS,-1,-1,0.0,0,0
22,1984,CALIFORNIA,-1,-1,0.0,0,0
...,...,...,...,...,...,...,...
188,2020,VIRGINIA,-1,-1,0.0,0,1
194,2020,WASHINGTON,-1,-1,0.0,0,1
164,2020,WEST VIRGINIA,-1,-1,0.0,0,1
198,2020,WISCONSIN,-1,-1,0.0,0,1


#### house dummy

In [22]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes', shift=2)
house_vote['candidatevotes'] = house_vote['candidatevotes'].apply(lambda x: -1 if x < 50 else 1)
house_vote = house_vote.rename(columns={'candidatevotes':'house_midterm'})
house_vote = house_vote.sort_values(by=['year', 'state'])
house_vote.to_csv('datasets-clean/incumbent-house-rep.csv', index=False)
#dflist.append(house_vote)
house_vote

,year,state,house_midterm
42,1984,ALABAMA,1
5,1984,ALASKA,-1
15,1984,ARIZONA,-1
27,1984,ARKANSAS,1
22,1984,CALIFORNIA,1
...,...,...,...
183,2020,VIRGINIA,1
189,2020,WASHINGTON,1
161,2020,WEST VIRGINIA,-1
193,2020,WISCONSIN,1


#### personal income

In [23]:
inc_cap = to_splatter(pd.read_csv('datasets/personal-income-per-capita-72-20.csv'), 'avg_inc')
inc_cap.avg_inc = inc_cap.avg_inc.apply(lambda x: math.log(x))
inc_cap = inc_cap.set_index(['year', 'state'])
inc_cap = beadiff(inc_cap, 'avg_inc')
inc_cap = var42(inc_cap, 'avg_inc', 'income-index', 'avg_inc')
dflist.append(inc_cap)
inc_cap

,year,state,avg_inc_4,avg_inc_3,avg_inc_2,avg_inc_1
43,1984,ALABAMA,0.987296,0.724065,0.580288,1.116090
5,1984,ALASKA,0.115255,0.028116,1.312881,1.007980
15,1984,ARIZONA,0.954184,0.744183,0.370798,1.166796
27,1984,ARKANSAS,1.122251,0.579421,0.541543,1.377405
22,1984,CALIFORNIA,0.899258,0.583583,0.496915,1.030214
...,...,...,...,...,...,...
188,2020,VIRGINIA,0.486027,0.277813,0.293510,0.294663
194,2020,WASHINGTON,0.571009,0.393115,0.459420,0.383196
164,2020,WEST VIRGINIA,0.535082,0.245300,0.503602,0.446958
198,2020,WISCONSIN,0.453397,0.295339,0.395814,0.306832


#### unenployment

In [24]:
unn = pd.read_csv('datasets/unemployment-76-21-percent.csv' if True else 'datasets/unemployment-us-states.csv')
furi = 'datasets-clean/serial-id-bsl.pkl'

if False:
    sd = dict()
    for s in set([*unn['Series ID'], *nunn['Series ID']]):
        try:
            sd[s] = bls_serial(s)
        except IndexError:
            sd[s] = f'BADASS_{s}'
    pk.dump(sd, open(furi, 'wb'))
else:
    sd = pk.load(open(furi, 'rb'))

#unn.Label = unn.Label.apply(lambda x : x.split()[-1])
unn = unn[unn.Period == 'M01']
unn['Series ID'] = unn['Series ID'].map(sd)
unn = unn.rename(columns={
    "Series ID": "state",
    "Year": "year",
    "Value": "unemp",
    })
unn['unemp'] = unn['unemp'].apply(foofloat).astype(float)
unn = unn.dropna()
#print(whore(unn.set_index(['year', 'state']), range(1990,2021)))
unn = var42(unn, 'unemp', 'unemployment-index', 'unemp')
dflist.append(unn)
unn

,year,state,unemp_4,unemp_3,unemp_2,unemp_1
43,1984,ALABAMA,12.3,14.9,12.7,9.7
5,1984,ALASKA,10.2,10.7,9.8,9.1
15,1984,ARIZONA,5.7,11.6,7.8,6.1
27,1984,ARKANSAS,8.8,10.1,9.3,8.2
22,1984,CALIFORNIA,8.3,11.1,8.8,7.1
...,...,...,...,...,...,...
188,2020,VIRGINIA,2.5,2.9,3.2,4.0
194,2020,WASHINGTON,4.0,4.5,4.5,4.8
164,2020,WEST VIRGINIA,5.0,5.0,5.4,5.4
198,2020,WISCONSIN,3.3,3.1,3.0,3.5


### merging features

In [25]:
df = pd.DataFrame(data=[[y,s] for y in years for s in states], columns=['year', 'state'])
for d in dflist:
    df = df.merge(d, how='inner', left_on=['year','state'], right_on=['year','state'])

df = df.sort_values(by=['year', 'state'])
df.to_csv('datasets-clean/xxx-final-dataset.csv', index=False)
df

,year,state,y_votes_percent,def_4,def_3,def_2,def_1,z_4,z_3,z_2,...,sudden_vice,lag_vice,avg_inc_4,avg_inc_3,avg_inc_2,avg_inc_1,unemp_4,unemp_3,unemp_2,unemp_1
43,1984,ALABAMA,38.736605,4.548485,4.185139,6.426162,9.372111,1,1,0,...,0,0,0.987296,0.724065,0.580288,1.116090,12.3,14.9,12.7,9.7
5,1984,ALASKA,30.944087,1.250442,-1.018197,4.758325,23.893438,1,0,0,...,0,0,0.115255,0.028116,1.312881,1.007980,10.2,10.7,9.8,9.1
15,1984,ARIZONA,32.883272,4.731524,5.112800,6.966784,8.376158,1,1,0,...,0,0,0.954184,0.744183,0.370798,1.166796,5.7,11.6,7.8,6.1
27,1984,ARKANSAS,38.772412,4.363585,4.161762,5.603037,8.995826,1,1,0,...,0,0,1.122251,0.579421,0.541543,1.377405,8.8,10.1,9.3,8.2
22,1984,CALIFORNIA,41.775465,4.737549,4.477916,6.742430,8.871767,1,1,0,...,0,0,0.899258,0.583583,0.496915,1.030214,8.3,11.1,8.8,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2020,VIRGINIA,55.154687,1.950355,2.222617,1.866446,1.307918,0,0,0,...,0,1,0.486027,0.277813,0.293510,0.294663,2.5,2.9,3.2,4.0
194,2020,WASHINGTON,59.925503,1.658615,1.894215,1.658639,1.276351,0,1,1,...,0,1,0.571009,0.393115,0.459420,0.383196,4.0,4.5,4.5,4.8
164,2020,WEST VIRGINIA,30.201468,-0.257632,0.908538,3.283808,3.326667,0,0,0,...,0,1,0.535082,0.245300,0.503602,0.446958,5.0,5.0,5.4,5.4
198,2020,WISCONSIN,50.319063,2.055099,2.222565,1.889366,1.230655,0,0,0,...,0,1,0.453397,0.295339,0.395814,0.306832,3.3,3.1,3.0,3.5


In [26]:
def whore(df, yy=years, ss=states):
    x = []
    for s in ss:
        for y in yy:
            try:
                df.loc[(y,s),:]
            except:
                x.append((y,s))
    return x

In [27]:
for d in dflist:
    print('-+'*30)
    print(len(whore(d.set_index(['year', 'state']))))
    print('-+'*30)
    print(d.head())

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
0
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
    year       state  y_votes_percent
43  1984     ALABAMA        38.736605
5   1984      ALASKA        30.944087
15  1984     ARIZONA        32.883272
27  1984    ARKANSAS        38.772412
22  1984  CALIFORNIA        41.775465
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
0
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
    year       state     def_4     def_3     def_2      def_1
43  1984     ALABAMA  4.548485  4.185139  6.426162   9.372111
5   1984      ALASKA  1.250442 -1.018197  4.758325  23.893438
15  1984     ARIZONA  4.731524  5.112800  6.966784   8.376158
27  1984    ARKANSAS  4.363585  4.161762  5.603037   8.995826
22  1984  CALIFORNIA  4.737549  4.477916  6.742430   8.871767
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
0
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
    year       state  z_4  z

In [28]:
0/0
# chernobyl zone

ZeroDivisionError: division by zero

exp fail

In [ ]:
gdp_boot = gdp_real.copy()
gdp_exp = gdp_boot.set_index(['year', 'state'])
for s in states:
    print(gdp_exp.loc[(slice(None),s),:].gdp_real.apply(lambda x : math.exp((x - gdp_exp.gdp_real.mean())/ math.sqrt(gdp_exp.gdp_real.var()))))
    break

us stuff

In [ ]:
gdp_us = pd.read_csv('datasets/gdp-nomina-47-20-chain-12.csv')
gdp_us.DATE = gdp_us.DATE.apply(lambda x: int(str(x).split('-')[0]))
s = gdp_us[gdp_us.DATE > 1975].GDPC1_PC1
s.hist()

In [ ]:
ecdf = ECDF(s)
ecdf(2.8)

qnt = 0.67
std_err = np.sqrt(s.var())/2
ecdf_inv(s, qnt), len(s)*(1-qnt), std_err

#### house

In [ ]:
'''
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })
#'''

#### gasoline and friends + approval

In [ ]:
gas = pd.read_csv('datasets/gasoline-93-21.csv')
gas.date = gas.date.apply(lambda x: time.mktime(datetime.datetime.strptime(x,"%m/%d/%Y").timetuple()))
gas.date = gas.date.apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
gas

#### houses prices and rent + personal income

In [ ]:
f = open('datasets/house-chain-00.csv', 'rt')
lines = f.readlines()
houses = [[i.strip() for i in l.split(sep = '$') ] for l in lines ]
houses = pd.DataFrame(data=houses, columns=['state', 2000, 1990, 1980, 1970, 1960, 1950, 1940 ])
houses.head()